# Bloque 1: Importaciones y Configuración

- Propósito: Cargar las librerías necesarias (numpy, librosa, joblib, os).
También definimos las variables clave:
1) MODELO_PATH: Dónde está guardado nuestro modelo entrenado.
2) AUDIO_PARA_CLASIFICAR: El nombre del archivo de audio que queremos probar.
3) CLASES_MAP: Un "diccionario" para traducir la predicción numérica (0, 1, 2) a texto.
4) N_MFCC: El número de características que usamos (debe ser el mismo del entrenamiento).

In [134]:
# Importaciones
import numpy as np
import librosa
import joblib
import os 
MODELO_PATH = "clasificador_vehiculos.joblib"
AUDIO_PARA_CLASIFICAR = "audio_random.wav" 

# Mapeo de las etiquetas numéricas a texto
CLASES_MAP = {0: "pequeno", 1: "mediano", 2: "grande"} 
N_MFCC = 40 


# Bloque 2: Función de Extracción de Características

- Propósito: Definir la función que convierte un archivo de audio en números.
Esta función DEBE ser una copia exacta de la que se usó en el notebook 1.
Carga el audio, calcula los 40 MFCCs, y luego calcula la media de esos MFCCs
para obtener un solo vector de características.

In [135]:
# Bloque 2: Función de Extracción de Características
def extract_features(file_path, n_mfcc=40):
    """Carga un archivo de audio (path) y extrae la media de 40 MFCCs."""
    try:
        y, sr = librosa.load(file_path, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        features_vector = np.mean(mfccs.T, axis=0)
    except Exception as e:
        print(f"Error cargando {file_path}: {e}")
        return None
    return features_vector

# Bloque 3: Carga del Modelo

- Propósito: Cargar en memoria el modelo .joblib que creamos en el notebook 1.
'joblib.load()' lee el archivo y lo convierte de nuevo en un objeto de Python
(nuestro clasificador 'model') que está listo para hacer predicciones.

In [136]:
# Bloque 3: Carga del Modelo
print(f"Cargando modelo desde: {MODELO_PATH}")
try:
    model = joblib.load(MODELO_PATH)
    print("¡Modelo cargado exitosamente!")
except FileNotFoundError:
    print(f"ERROR: No se encontró el archivo '{MODELO_PATH}'.")
    print("Asegúrate de ejecutar el Notebook '01_Entrenamiento_Modelo.ipynb' primero.")
    raise

Cargando modelo desde: clasificador_vehiculos.joblib
¡Modelo cargado exitosamente!


# Bloque 4: Clasificación del Audio

- Propósito: Usar el modelo

1) Verifica si el archivo de audio existe.
2) Llama a la función 'extract_features' (del Bloque 2) para convertir el audio en un vector.
3) Usa 'model.predict()' para obtener la predicción (que será 0, 1 o 2).
4) Usa 'CLASES_MAP' (del Bloque 1) para traducir el número a "pequeno", "mediano" o "grande".
5) Muestra el resultado final al usuario.

In [137]:
# Bloque 4: Clasificación del Audio
print(f"\nAnalizando audio: {AUDIO_PARA_CLASIFICAR}...")

# Verificar que el audio exista
if not os.path.exists(AUDIO_PARA_CLASIFICAR):
    print(f"ERROR: No se encontró el archivo de audio '{AUDIO_PARA_CLASIFICAR}'.")
else:
    # Extraer características del audio (usando la misma función)
    features = extract_features(AUDIO_PARA_CLASIFICAR, n_mfcc=N_MFCC)
    
    if features is not None:
        # Preparar el vector para el modelo
        features_2d = features.reshape(1, -1)
        
        # Predecir
        prediccion_num = model.predict(features_2d)
        
        # Obtener el resultado en texto
        clase_predicha = CLASES_MAP[prediccion_num[0]]
        
        # Mostrar el resultado
        print("\n===============================")
        print("     ✅ RESULTADO ✅")
        print("===============================")
        print(f"El audio '{AUDIO_PARA_CLASIFICAR}' ha sido clasificado como:")
        print(f" -> **{clase_predicha.upper()}**")
        print("===============================")
    else:
        print("No se pudieron extraer características del audio.")


Analizando audio: audio_random.wav...

     ✅ RESULTADO ✅
El audio 'audio_random.wav' ha sido clasificado como:
 -> **GRANDE**


EXPLIACION

In [138]:
import numpy as np
import librosa
import joblib
import os 

MODELO_PATH = "clasificador_vehiculos.joblib"
AUDIO_PARA_CLASIFICAR = "audio_random.wav" 

CLASES_MAP = {0: "pequeno", 1: "mediano", 2: "grande"} 
N_MFCC = 40 


SEGMENT_DUR_SEC = 0.5  # Duración de la ventana de análisis (ej: 1 segundo)
OVERLAP_RATIO = 0.8    # Solapamiento entre ventanas (50%)
MIN_EVENT_SEGMENTS = 5 # Mínimo de segmentos consecutivos para contar como evento único

EXPLICACION

In [139]:
def extract_features(file_path, n_mfcc=40):
    try:
        # Si file_path es una ruta (string), cargamos. 
        # Si ya es un array de audio (segmento), usamos lo que viene.
        if isinstance(file_path, str):
             y, sr = librosa.load(file_path, res_type='kaiser_fast')
        else:
             # Caso especial para cuando pasas el segmento directo en el conteo
             y = file_path
             sr = 22050 # Valor por defecto de librosa si no se tiene sr
        
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        
        # --- MEJORA AQUÍ ---
        # Calculamos la media Y la desviación estándar
        mfccs_mean = np.mean(mfccs.T, axis=0)
        mfccs_std = np.std(mfccs.T, axis=0)
        
        # Concatenamos ambos para tener un vector de 80 números (40 media + 40 std)
        features_vector = np.hstack([mfccs_mean, mfccs_std])
        # -------------------
        
    except Exception as e:
        print(f"Error: {e}")
        return None
    return features_vector

EXPLICACION

In [140]:
print(f"\nAnalizando audio largo: {AUDIO_PARA_CLASIFICAR}...")

# 1. Verificar y Cargar el Audio Completo
if not os.path.exists(AUDIO_PARA_CLASIFICAR):
    print(f"ERROR: No se encontró el archivo de audio '{AUDIO_PARA_CLASIFICAR}'.")
else:
    # Cargar el modelo (Asume que el Bloque 3 ya cargó 'model')
    try:
        model = joblib.load(MODELO_PATH)
    except Exception as e:
        print(f"Error al cargar el modelo: {e}. Asegúrese de ejecutar el notebook de entrenamiento.")
        exit()

    # Cargar el audio completo
    y, sr = librosa.load(AUDIO_PARA_CLASIFICAR, res_type='kaiser_fast') 
    
    # 2. Configurar y Ejecutar la Ventana Deslizante
    segment_len = int(SEGMENT_DUR_SEC * sr)    # Longitud de la ventana en muestras
    hop_len = int(segment_len * (1 - OVERLAP_RATIO)) # Paso para el solapamiento

    features_list = []
    
    # Iterar y extraer características de cada segmento
    for i in range(0, len(y) - segment_len, hop_len):
        y_segment = y[i:i + segment_len]
        # NOTA: Usamos la función adaptada para segmentos
        features = extract_features_segment(y_segment, sr, n_mfcc=N_MFCC)
        
        if features is not None:
            features_list.append(features)

    if not features_list:
        print("No se pudieron extraer segmentos del audio (el archivo es muy corto o hubo un error).")
    else:
        # 3. Predecir todos los segmentos
        X_segments = np.array(features_list)
        predicciones = model.predict(X_segments) # Array largo de predicciones [0, 0, 1, 1, 1, ...]

        # 4. Lógica de Conteo de Eventos Únicos (Agregación)
        conteo_final = {clase: 0 for clase in CLASES_MAP.values()}
        
        event_class = -1
        consecutive_count = 0
        
        for pred in predicciones:
            if pred == event_class:
                # El evento continúa
                consecutive_count += 1
            else:
                # El evento terminó o es el inicio de uno nuevo
                if event_class != -1 and consecutive_count >= MIN_EVENT_SEGMENTS:
                    # El evento anterior fue válido y se cuenta
                    clase_texto = CLASES_MAP[event_class]
                    conteo_final[clase_texto] += 1
                
                # Reiniciar el conteo para la nueva clase
                event_class = pred
                consecutive_count = 1
        
        # Comprobar el último evento al salir del bucle
        if event_class != -1 and consecutive_count >= MIN_EVENT_SEGMENTS:
            clase_texto = CLASES_MAP[event_class]
            conteo_final[clase_texto] += 1
            
        # 5. Mostrar el resultado
        print("\n===============================")
        print("      RESULTADO FINAL DE CONTEO")
        print("===============================")
        print(f"Audio analizado: {AUDIO_PARA_CLASIFICAR} ({len(y)/sr:.2f} segundos)")
        print(f"Total de segmentos procesados: {len(features_list)}")
        print(f"Duración mínima de evento para conteo: {MIN_EVENT_SEGMENTS} segmentos.")
        print("---")
        
        for clase, count in conteo_final.items():
            print(f"Vehículos tipo **{clase.upper()}**: {count} detectados")
            
        print("===============================")


Analizando audio largo: audio_random.wav...

      RESULTADO FINAL DE CONTEO
Audio analizado: audio_random.wav (13.47 segundos)
Total de segmentos procesados: 130
Duración mínima de evento para conteo: 5 segmentos.
---
Vehículos tipo **PEQUENO**: 1 detectados
Vehículos tipo **MEDIANO**: 5 detectados
Vehículos tipo **GRANDE**: 1 detectados


EXPLICACION